In [2]:
import pandas as pd
from tqdm.auto import tqdm
from pycparser import parse_file
from pycparser.plyparser import ParseError
import numpy as np
import os
import clang.cindex
import subprocess
import pickle

In [5]:
codes = pd.read_csv('../Datasets Code Snippets/post_with_only_code_snippets_final.csv')

In [6]:
codes

,id,code
0,37637877,void closeBuySell(int buyTicket){\n double b...
1,57596800,"#include <stdio.h>\n\nvoid sum(int *, int *, i..."
2,22667586,"#include ""buraco/game/board/model/KingChatFilt..."
3,22667586,"#include ""buraco\game/board/model\CardGroup.h""..."
4,55014685,#include <stdlib.h> /* EXIT malloc free */\n#i...
...,...,...
452666,64735134,"template <class T, class ...Ts>\nvoid printA(T..."
452667,65106538,#include <iostream>\nusing namespace std;\n\nv...
452668,25117355,#include <iostream>\nusing namespace std;\n\ni...
452669,55782153,#include<stdio.h>\n#include <limits.h>\n#inclu...


# Filter codes by keywords

In [3]:
words_to_search = ['strcpy', 'strncpy', 'strcat', 'strncat', 'system', 'memcpy', 'memset', 'malloc', 'gets', 'vfork', 
                   'realloc', 'pthread_mutex_lock', 'free', 'chroot', 'strlen', 'vsprintf', 'sprintf', 'scanf', 'fscanf', 
                   'sscanf', 'vscanf', 'vsscanf', 'vfscanf','snprintf', 'atoi', 'strtok', 'strcmp', 'strncmp', 'strcasecmp',
                    'strncasecmp', 'memcmp', 'signal', 'va_arg', 'vulnerab', 'fault', 'defect', 'sanit', 'mistake', 'flaw', 
                   'bug', 'infinite', 'loop', 'secur', 'overflow', 'error', 'remote', 'mitigat', 'realloc', 'heap', 
                   'privilege', 'underflow', 'attack', 'DoS', 'denial-of-service', 'initiali', 'xss', 'leak', 'patch',
                    'authori', 'corruption', 'crash', 'memory', 'null', 'injection', 'out-of-bounds', 'use-after-free', 
                   'dereferenc', 'buffer', 'hack', 'segment', 'authentication', 'exploit', 'incorrect', 'vulnerab', 'harm', 
                   'undefine', 'unpredict', 'unsafe', 'secur', 'malicious', 'dangerous', 'critical', 'bad', 'unprivileged', 
                   'negative', 'stable', 'invalid', 'flaw', 'hack', 'fix', 'change', 'modify', 'exploit', 'mitigat', 'leak', 
                   'realloc', 'invoke', 'inject', 'ensure', 'reject', 'initiali', 'fail', 'authori', 'update', 'attack', 
                   'trigger', 'lock', 'corrupt', 'crash', 'prevent', 'avoid', 'access', 'cause', 'overflow', 'terminat'              
                  ]

In [6]:
dict_keywords = dict()
idx = 0

for word in tqdm(words_to_search):
    dict_word= dict()
    for index in range(codes.shape[0]):
        if word in codes['code'][index]:
            dict_word[idx]={"id": codes['id'][index], "code": codes['code'][index]}
            idx=idx+1
    
    print(len(dict_word), " codes contain word ",word)
    dict_keywords = {**dict_keywords, **dict_word}

9869  codes contain word  strcpy
1907  codes contain word  strncpy
2749  codes contain word  strcat
281  codes contain word  strncat
13871  codes contain word  system
5841  codes contain word  memcpy
6992  codes contain word  memset
30558  codes contain word  malloc
14012  codes contain word  gets
43  codes contain word  vfork
3766  codes contain word  realloc
1249  codes contain word  pthread_mutex_lock
22123  codes contain word  free
7  codes contain word  chroot
16676  codes contain word  strlen
117  codes contain word  vsprintf
4792  codes contain word  sprintf
32076  codes contain word  scanf
4363  codes contain word  fscanf
2475  codes contain word  sscanf
1  codes contain word  vscanf
7  codes contain word  vsscanf
15  codes contain word  vfscanf
1767  codes contain word  snprintf
5058  codes contain word  atoi
3465  codes contain word  strtok
7427  codes contain word  strcmp
818  codes contain word  strncmp
184  codes contain word  strcasecmp
31  codes contain word  strncasecmp

In [7]:
len(dict_keywords)

481800

In [8]:
df_dict_keywords = pd.DataFrame.from_dict(dict_keywords, orient='index')

In [9]:
df_dict_keywords = df_dict_keywords.drop_duplicates().reset_index(drop=True)

In [10]:
df_dict_keywords

,id,code
0,30877241,void list_file(char* directory) {\n DIR *d;\n...
1,40233070,/***** PLEASE DO NOT EVER DO THIS IN REAL CO...
2,29652839,#include <Windows.h>\n#include <string>\n\nboo...
3,55118246,#include <cstdlib>\n#include <cstring>\n#inclu...
4,53767640,void dir_details(int socket) {\n //Set up a...
...,...,...
219569,29433869,void MainWindow::Quit()\n{\n close();\n ...
219570,49517511,#include <iostream>\n#include <string>\n#inclu...
219571,22337006,#include <iostream>\n#include<stdio.h>\n#inclu...
219572,48448538,"#include ""stdafx.h""\n#include < iostream>\n#in..."


In [13]:
df_dict_keywords.to_csv('../Datasets Code Snippets/code_snippets_static_analysis_v2.csv', index=False)

# Detecting vulnerabilities

In [3]:
df_files_static_analysis = pd.read_csv('../Datasets Code Snippets/code_snippets_static_analysis_v2.csv')
df_files_static_analysis

,id,code
0,30877241,void list_file(char* directory) {\n DIR *d;\n...
1,40233070,/***** PLEASE DO NOT EVER DO THIS IN REAL CO...
2,29652839,#include <Windows.h>\n#include <string>\n\nboo...
3,55118246,#include <cstdlib>\n#include <cstring>\n#inclu...
4,53767640,void dir_details(int socket) {\n //Set up a...
...,...,...
219569,29433869,void MainWindow::Quit()\n{\n close();\n ...
219570,49517511,#include <iostream>\n#include <string>\n#inclu...
219571,22337006,#include <iostream>\n#include<stdio.h>\n#inclu...
219572,48448538,"#include ""stdafx.h""\n#include < iostream>\n#in..."


In [3]:
for index in tqdm(range(df_files_static_analysis.shape[0])):
    file_name = f"{index}.cpp"
    with open("code_static_analysis/"+file_name, "w", encoding="utf-8") as f:
        f.write(df_files_static_analysis.iloc[index, 1])

In [3]:
result_static_analysis = dict()
index = 0

1539

'82712.cpp'

In [37]:
already_checked = []

with open("static_analysis.pkl", "rb") as f:
    result_static_analysis = pickle.load(f)
    index = list(result_static_analysis)[-1]+1

for key in result_static_analysis.keys():
    already_checked.append(result_static_analysis[key]['file'])
    
print(len(already_checked))

print(already_checked[-1])

to_check = set(os.listdir('code_static_analysis')) - set(already_checked)
print(len(to_check))

for file in tqdm(sorted(to_check)):
    try:
        #print(file)
        i = int(file.split('.')[0])
        proc = subprocess.run(["cppcheck", "--enable=style", "--template={cwe}:{file}:{line}:{severity}:{code}:{message}", "code_static_analysis/"+file], capture_output=True)
        result = proc.stderr.decode('utf-8')
        if result:
            items = result.split('\r\n')
            items = list(filter(None, items))

            for x, y in zip(*[iter(items)] * 2):
                row = x + y
                row = row.split(':')
                result_static_analysis[index]={'id_post': df_files_static_analysis.iloc[i, :]['id'], 'cwe':row[0], 'file': file, 'line': row[2], 'severity': row[3],
                                      'code': row[4], 'message': row[5]}
                index = index + 1
        else:
            result_static_analysis[index]={'id_post': df_files_static_analysis.iloc[i, :]['id'], 'cwe':0, 'file': file, 'line': '', 'severity': '',
                                      'code': '', 'message': ''}
            index = index + 1
        
    except UnicodeDecodeError as e:
        with open("static_analysis.pkl", "wb") as f:
            pickle.dump(result_static_analysis, f)
        print(e)
        print(file)
        print(proc.stderr)
        break;

388570
99964.cpp
46


In [38]:
with open("static_analysis.pkl", "wb") as f:
    pickle.dump(result_static_analysis, f)

In [39]:
len(result_static_analysis)

388648

In [40]:
df_result_static_analysis = pd.DataFrame.from_dict(result_static_analysis, orient='index')
df_result_static_analysis

,id_post,cwe,file,line,severity,code,message
0,30877241,398,0.cpp,3,style,struct dirent *dir; ^,The scope of the variable 'dir' can be reduced.
1,40233070,0,1.cpp,27,warning,char *buf = alloca(*bufsize + 128); ...,Obsolete function 'alloca' called.
2,40233070,0,1.cpp,46,warning,char *claim_the_space = alloca(bufsize + 2...,Obsolete function 'alloca' called.
3,20898874,398,10.cpp,11,style,data = (struct snail_thread *) ptr; ...,C-style pointer casting
4,29069387,758,100.cpp,60,error,os<<input.display(&os); ^,Found a exit path from function with non-void ...
...,...,...,...,...,...,...,...
388643,47004715,457,99996.cpp,25,warning,"printf(""%d"", total); ^",Uninitialized variable
388644,47004715,457,99996.cpp,21,error,total++; ^,Uninitialized variable
388645,34116859,0,99997.cpp,,,,
388646,55820831,0,99998.cpp,,,,


In [41]:
print("Posts without vulnerabilities: " )
df_result_static_analysis.loc[df_result_static_analysis['severity']=='']

Posts without vulnerabilities: 


,id_post,cwe,file,line,severity,code,message
7,34004371,0,10000.cpp,,,,
23,46909578,0,100002.cpp,,,,
25,60496904,0,100004.cpp,,,,
32,43029254,0,100009.cpp,,,,
34,61317474,0,100010.cpp,,,,
...,...,...,...,...,...,...,...
388633,37308917,0,99993.cpp,,,,
388640,43673484,0,99995.cpp,,,,
388645,34116859,0,99997.cpp,,,,
388646,55820831,0,99998.cpp,,,,


In [43]:
severities = pd.unique(df_result_static_analysis['severity'])
severities = list(filter(None, severities))

for severity in severities:
    to_delete = df_result_static_analysis.loc[df_result_static_analysis['cwe']=='0'].loc[df_result_static_analysis['severity']==severity].index
    #df_result_static_analysis = df_result_static_analysis.drop(to_delete)
    print(len(to_delete))
#df_result_static_analysis


1811
157
44150
52
45
6


In [44]:
print("Count by severity:")

for severity in severities:
    print(severity+": ",df_result_static_analysis.loc[df_result_static_analysis['severity']==severity].count()[0])
print("Without vulnerabilities: ",df_result_static_analysis.loc[df_result_static_analysis['severity']==''].count()[0])

Count by severity:
style:  173004
error:  71834
portability:  6003
performance:  6477
debug:  6
Without vulnerabilities:  91339


In [45]:
df_result_static_analysis.to_csv('Results/results_static_analysis_v2.csv', index=False)

In [46]:
df_result_static_analysis

,id_post,cwe,file,line,severity,code,message
0,30877241,398,0.cpp,3,style,struct dirent *dir; ^,The scope of the variable 'dir' can be reduced.
1,40233070,0,1.cpp,27,warning,char *buf = alloca(*bufsize + 128); ...,Obsolete function 'alloca' called.
2,40233070,0,1.cpp,46,warning,char *claim_the_space = alloca(bufsize + 2...,Obsolete function 'alloca' called.
3,20898874,398,10.cpp,11,style,data = (struct snail_thread *) ptr; ...,C-style pointer casting
4,29069387,758,100.cpp,60,error,os<<input.display(&os); ^,Found a exit path from function with non-void ...
...,...,...,...,...,...,...,...
388643,47004715,457,99996.cpp,25,warning,"printf(""%d"", total); ^",Uninitialized variable
388644,47004715,457,99996.cpp,21,error,total++; ^,Uninitialized variable
388645,34116859,0,99997.cpp,,,,
388646,55820831,0,99998.cpp,,,,


In [4]:
prova = pd.read_csv('Results/results_static_analysis_v2.csv')
prova.loc[prova['id_post']==44475317]

,id_post,cwe,file,line,severity,code,message
33692,44475317,398,118350.cpp,23.0,warning,Struct() = default; ^,Member variable 'foo < int >
33693,44475317,398,118350.cpp,23.0,warning,Struct() = default; ^,Member variable 'bar < double >


In [5]:
prova.loc[prova['id_post']==21646467]

,id_post,cwe,file,line,severity,code,message
24835,21646467,398,112792.cpp,42.0,style,"((App*) qApp)->invokeGenericExec(func, Qt",NaN
24836,21646467,398,112792.cpp,48.0,style,"((App*) qApp)->invokeGenericExec(func, Qt",NaN


In [7]:
prova.loc[prova['id_post']==26083517]

,id_post,cwe,file,line,severity,code,message
15051,26083517,686,107452.cpp,29.0,warning,"printf ("" number","%lu invalid (no digits found, 0 returned)\n..."
15052,26083517,686,107452.cpp,31.0,warning,"printf ("" number","%lu invalid (underflow occurred)\n"", number..."
15053,26083517,686,107452.cpp,33.0,warning,"printf ("" number","%lu invalid (overflow occurred)\n"", number)..."
15054,26083517,686,107452.cpp,35.0,warning,"printf ("" number",%lu invalid (base contains unsupported valu...
15055,26083517,686,107452.cpp,37.0,warning,"printf ("" number","%lu invalid (unspecified error occurred)\n""..."
15056,26083517,686,107452.cpp,39.0,warning,"printf ("" number",%lu valid (and represents all characters ...
15057,26083517,686,107452.cpp,41.0,warning,"printf ("" number",%lu valid (but additional characters rema...
15058,26083517,476,107452.cpp,22.0,warning,"number = strtol (nptr, &endptr, base ); ...",Either the condition 'nptr' is redundant or th...
15059,26083517,476,107452.cpp,25.0,warning,"printf (""\n string",%s\n base


In [9]:
prova.loc[prova['id_post']==13504934]

,id_post,cwe,file,line,severity,code,message
100447,13504934,398,157301.cpp,1.0,style,"void initialize_server(int port, int connectio...",Parameter 'address' can be declared as pointer...


In [11]:
from utils.functions import parse

In [23]:
with open('code_static_analysis/157301.cpp') as f:
    data = f.read()
    f.close()

result = parse.tokenizer(data)

In [26]:
len(result)

173